# Goal

to fetch the events from the calendar and edit them as a timebox given prompt and then write them back.

That way we can give this as a tool to the schedular agent as well, so it can change a single event, but also reschedule the day as a whole if needed.

In [1]:
from datetime import datetime, timedelta
work_start_hour = 8 # TODO: move to settings
is_before_work_start = datetime.now().time() < datetime.now().replace(hour=work_start_hour, minute=0, second=0, microsecond=0).time()
# is_before_work_start = True
today = datetime.now().date()
day_to_plan = today if is_before_work_start else today + timedelta(days=1)
day_to_plan

datetime.date(2026, 2, 3)

# now pull events from calendar for that date as a timebox object

```mermaid

flowchart TB
  node_1("CalendarEvent")

```

In [2]:
from dataclasses import dataclass
from datetime import datetime, date as date_type, time as time_type, timedelta, timezone
from typing import Any, Iterable
from zoneinfo import ZoneInfo

from dateutil import parser as date_parser

from fateforger.agents.schedular.models.calendar import CalendarEvent, EventType
from fateforger.agents.timeboxing.timebox import Timebox


from autogen_ext.tools.mcp import McpWorkbench, StreamableHttpServerParams

from fateforger.core.config import settings


# TODO: wrao this in a nice function

def calendar_workbench() -> McpWorkbench:

    return McpWorkbench(
        StreamableHttpServerParams(url=settings.mcp_calendar_server_url, timeout=10.0)
    )
workbench = calendar_workbench()

# query the workbench
result = await workbench.call_tool(
    "list-events", # TODO: what are all the args we can pass here?
    arguments={
        "calendarId": "primary",
        "timeMin": "2026-01-27T00:00:00Z",
        "timeMax": "2026-01-28T00:00:00Z",
        "singleEvents": True,
        "orderBy": "startTime",
    },
)

raw_json_string = result.result[0].content

# def get_timebox_for_day(day: date_type, timezone_str: str) -> Timebox:

In [3]:
result.result[0].content

'{"events":[{"id":"30287b8c333b44c18b3bfbd52f357d4f","summary":"morning review","start":{"dateTime":"2026-01-27T13:00:00+01:00","timeZone":"Europe/Amsterdam"},"end":{"dateTime":"2026-01-27T13:15:00+01:00","timeZone":"Europe/Amsterdam"},"status":"confirmed","htmlLink":"https://www.google.com/calendar/event?eid=MzAyODdiOGMzMzNiNDRjMThiM2JmYmQ1MmYzNTdkNGYgaHVnby5ldmVyc0Bt","created":"2026-01-26T21:07:30.000Z","updated":"2026-01-27T10:54:23.636Z","colorId":"10","creator":{"email":"hugo.evers@gmail.com","self":true},"organizer":{"email":"hugo.evers@gmail.com","self":true},"iCalUID":"30287b8c333b44c18b3bfbd52f357d4f@google.com","sequence":1,"reminders":{"useDefault":true},"eventType":"default","guestsCanModify":true,"calendarId":"primary","accountId":"normal"}],"totalCount":1}'

In [4]:
from fateforger.adapters.calendar.models import GCalEventsResponse

gcal_events = GCalEventsResponse.model_validate_json(result.result[0].content)

In [5]:
gcal_events

GCalEventsResponse(events=[GCalEvent(id='30287b8c333b44c18b3bfbd52f357d4f', summary='morning review', start=GCalEventDateTime(date_time='2026-01-27T13:00:00+01:00', date=None, time_zone='Europe/Amsterdam'), end=GCalEventDateTime(date_time='2026-01-27T13:15:00+01:00', date=None, time_zone='Europe/Amsterdam'), status='confirmed', html_link='https://www.google.com/calendar/event?eid=MzAyODdiOGMzMzNiNDRjMThiM2JmYmQ1MmYzNTdkNGYgaHVnby5ldmVyc0Bt', created='2026-01-26T21:07:30.000Z', updated='2026-01-27T10:54:23.636Z', creator=GCalPerson(email='hugo.evers@gmail.com', self_=True), organizer=GCalPerson(email='hugo.evers@gmail.com', self_=True), ical_uid='30287b8c333b44c18b3bfbd52f357d4f@google.com', sequence=1, reminders=GCalReminders(use_default=True, overrides=None), event_type='default', guests_can_modify=True, calendar_id='primary', account_id='normal', colorId='10')], total_count=1)

what is the goal here?
we want to give the agent a tool so it can pull the timebox for a given day.

we want to take that json object and turn it into a pydantic object.

# TODO: link to code

now we want to have a model that we use for timeboxing, so we convert it into a timebox object, l

In [6]:
# import from src/fateforger/adapters/calendar/models.py


In [7]:
from fateforger.agents.schedular.models.calendar import CalendarEvent



In [8]:
# now let the agent do it
# agent needs to know which day to fetch and the timezone, it can default to users.info for the timezone.
# and assume the user wants to plan for the next work day if before work start hour, unless the user specifies otherwise.


# Inspecting the timebox object

does it have all the properties/aliases to match the calendar event objects so we can mix and match directly?
Does the timebox object have the serialisation methods to inject it into the agents' context?

# now apply json patches

for this part we must determine what to prompt the model, so its going to generate json patches, but in which model? it needs to use the minimized schema, it will need to know about the current timebox object, it needs instructions, the inputs.

initially we dont want it to use issue json patches to create a timebox, instead it should print in markdown a rough draft of the timebox, which we can then convert to json patches to create the timebox object.
the question is if we assume the user agreees with the rought draft whether we can create the first real timebox.

In [9]:
# the timeboxing agent should have as it memory the timebox object, the inputs, and the last user message

# stage 3: rough draft of timebox

## TODO: 
* make sure the constraints and inputs are passed correctly to the rough draft agent, this means also injecting the current timebox events as fixed events.
* parametrize the planning rules and draft algorithm
* validate the markdown output and the planning trace output
* make sure we can insert this is toon input into the stage 4 timebox json patching agent

```xml
<SystemPrompt>
  <Role>Rough-Draft Day Planner Agent</Role>

  <Objective>
    Generate a bird’s-eye, glanceable rough draft of a day plan from user constraints and inputs.
    The output is meant to be refined in later stages; prioritize clarity, order, and approximate durations.
  </Objective>

  <Inputs>
    <Constraints>
      Fixed events with exact start/end (sleep, meetings, travel).
      Day start time (e.g., work starts 08:00).
      Preferred block sizes (e.g., DW=2h, half-DW=1h, quarter-DW=30m).
      Preferences (e.g., gym early, stimulant window ~4h).
    </Constraints>
    <Tasks>
      DailyOneThing (single most important outcome).
      Supporting tasks (meal prep, chores, fun/creative time).
      Optional constraints about stimulant timing, energy, or sequencing.
    </Tasks>
  </Inputs>

  <PlanningRules>
    <Rule id="R1">Honor all fixed-time constraints exactly; do not overlap events.</Rule>
    <Rule id="R2">Place DailyOneThing into 2–3 Deep Work blocks early-to-midday unless constraints prevent it.</Rule>
    <Rule id="R3">Default DW blocks to the user’s standard duration; allow 1h or 30m “partial DW” if specified.</Rule>
    <Rule id="R4">Insert buffers where risk is high: after DW (10–15m), after gym (30–45m reset), after meals (10–20m).</Rule>
    <Rule id="R5">Keep Shallow Work ≤30% of waking hours; cluster it later in the day when possible.</Rule>
    <Rule id="R6">Protect recovery and fun/creative blocks as first-class events; do not label them “optional” unless user did.</Rule>
    <Rule id="R7">If stimulants are mentioned: align demanding DW within the effective window; avoid back-to-back dosing unless user insists.</Rule>
    <Rule id="R8">Prefer simple sequencing over precision. This is a rough draft for later refinement.</Rule>
  </PlanningRules>

  <DraftAlgorithm>
    <Step>Lock fixed anchors (sleep, immovable meetings, hard start time).</Step>
    <Step>Place Morning Routine immediately after sleep end (15–45m, per user preference).</Step>
    <Step>Schedule DW blocks for DailyOneThing: hardest block first, then execution/polish blocks.</Step>
    <Step>Place gym in the user’s preferred slot (e.g., early) and add a reset buffer after.</Step>
    <Step>Place a fun/unstructured block (1–2h) if requested, ideally between stimulant windows or after a major block.</Step>
    <Step>Place shallow work (chores/admin) later; keep it bounded.</Step>
    <Step>Place meal prep + dinner in the evening; then creative time if requested.</Step>
    <Step>End with shutdown ritual + reading in bed + sleep anchor.</Step>
  </DraftAlgorithm>

  <OutputSpec>
    <PrimaryOutput format="markdown">
      <![CDATA[
      Produce a short, glanceable summary with:
      - A few headings (Night/Morning/Midday/Afternoon/Evening) OR emojis (optional).
      - ONE line per major chunk (not per micro-event).
      - Each line: **Label** — rough duration (e.g., "Deep Work (primary) — 2h").
      - Do not include exact timestamps unless the user explicitly asked for times.
      - Keep total output under ~12–15 lines.
      ]]>
    </PrimaryOutput>

    <PlanningTrace format="xml">
      <TraceRules>
        <Rule>Do NOT include chain-of-thought or detailed internal reasoning.</Rule>
        <Rule>Include only: assumptions, placements, durations, and checks.</Rule>
      </TraceRules>
      <TraceTemplate>
        <![CDATA[
        <PlanTrace>
          <Assumptions>
            <Assumption id="A1">...</Assumption>
          </Assumptions>
          <Placements>
            <Placement type="DW" label="DailyOneThing" duration="PT2H" rationale="hardest first"/>
            <Placement type="H" label="Gym" duration="PT1H30M" rationale="early preference"/>
          </Placements>
          <Checks>
            <Check name="NoOverlaps" status="pass"/>
            <Check name="ShallowWorkRatio" status="pass|warn" value="..."/>
            <Check name="DWCount" status="pass|warn" value="..."/>
          </Checks>
        </PlanTrace>
        ]]>
      </TraceTemplate>
    </PlanningTrace>

    <Tone>
      Crisp, minimal, readable at a glance. No motivational speeches.
    </Tone>
  </OutputSpec>

  <FailureModes>
    <Mode id="F1">Missing fixed anchors: infer a reasonable sleep block only if user provided sleep duration + bedtime; otherwise ask one question.</Mode>
    <Mode id="F2">Too many tasks: prioritize DailyOneThing and cap DW at 3 blocks; push the rest to shallow work or mark as overflow.</Mode>
    <Mode id="F3">Timing conflict: preserve fixed events, then reflow flexible blocks; keep buffers if possible.</Mode>
  </FailureModes>
</SystemPrompt>

```

In [10]:
from llama_index.tools.artifact_editor import (
    ArtifactEditorToolSpec,
    ArtifactMemoryBlock,
)

In [11]:
from openai import OpenAI
from dotenv import load_dotenv
import os
try:
    load_dotenv()
except AssertionError:
    load_dotenv(dotenv_path=".env")


token = os.getenv("OPENROUTER_API_KEY")
open_router_base_url = os.getenv("OPENROUTER_BASE_URL")
client = OpenAI(api_key=token, base_url=open_router_base_url)


In [12]:
from pydantic import BaseModel
from typing import Literal, Optional

class PizzaOrder(BaseModel):
    size: Literal["small", "medium", "large", "extra-large"]
    toppings: list[str]
    address: str


schema = PizzaOrder.model_json_schema()

prompt = lambda order: f"""
You are an expert pizza ordering agent. You need to order a pizza for the user based on their preferences.
you respond with json and list the size, toppings and address to deliver to.
Here is the order:
{order}
"""

order = "I want a large pizza with pepperoni and mushrooms, deliver to 123 Main St."
model="google/gemini-3-flash-preview"
response = client.responses.create(
  model=model,
  input=[{"role": "user", "content": prompt(order)}],
  text={
    "format": {
      "type": "json_schema",
      "name": "shopping_list",
      "strict": True,
      "schema": schema
    }
  }
)
pizza_order = PizzaOrder.model_validate_json(response.output_text)
pizza_order

PizzaOrder(size='large', toppings=['pepperoni', 'mushrooms'], address='123 Main St.')

In [13]:
import json
from typing import Annotated, Generic, Literal, Optional, Type, TypeVar, Union, get_args, get_origin

from pydantic import BaseModel, Field, create_model, model_validator
from llama_index.tools.artifact_editor import ArtifactEditorToolSpec
from llama_index.tools.artifact_editor.base import JsonPatch
from jinja2 import Template

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool

from fateforger.llm import build_autogen_chat_client


Style = Literal["napoletana", "new-york", "sicilian", "detroit", "roman"]
Size = Literal["s", "m", "l", "xl"]


class Pizza(BaseModel):
    style: Style
    toppings: str
    size: Size
    quantity: int = Field(ge=1)


class PizzaOrder(BaseModel):
    pizzas: list[Pizza] = Field(default_factory=list)


T = TypeVar("T", bound=BaseModel)


ErrorCategory = Literal[
    "invalid_path",
    "invalid_index",
    "invalid_enum",
    "missing_field",
    "schema_validation",
    "tool_call_failure",
    "patch_of_patch_missing",
    "unknown",
]


class ErrorInfo(BaseModel):
    category: ErrorCategory
    message: str
    op: dict | None = None
    op_index: int | None = None


class PatchResult(BaseModel, Generic[T]):
    ok: bool
    result: T
    current_artifact: T
    applied: list[dict] = Field(default_factory=list)
    error: str | None = None
    summary: str | None = None
    error_info: ErrorInfo | None = None


def _unwrap_optional(annotation: Type) -> Type:
    origin = get_origin(annotation)
    if origin is Union:
        args = [arg for arg in get_args(annotation) if arg is not type(None)]
        if len(args) == 1:
            return args[0]
    return annotation


def _infer_list_field(model_cls: Type[BaseModel]) -> str:
    # General-purpose: expect exactly one list field unless overridden.
    fields = getattr(model_cls, "model_fields", None) or getattr(model_cls, "__fields__", {})
    list_fields = []
    for name, field in fields.items():
        annotation = getattr(field, "annotation", None) or getattr(field, "outer_type_", None)
        annotation = _unwrap_optional(annotation)
        if get_origin(annotation) is list:
            list_fields.append(name)
    if len(list_fields) != 1:
        raise ValueError(f"Expected exactly one list field, found: {list_fields}")
    return list_fields[0]


def _infer_item_model(model_cls: Type[T], list_field: str) -> Type:
    field = (getattr(model_cls, "model_fields", None) or getattr(model_cls, "__fields__", {}))[list_field]
    annotation = getattr(field, "annotation", None) or getattr(field, "outer_type_", None)
    annotation = _unwrap_optional(annotation)
    args = get_args(annotation)
    if not args:
        raise ValueError(f"List field {list_field} is missing item type")
    return args[0]


def make_patch_schema(item_model: Type) -> Type[BaseModel]:
    # Typed patch schema: add/replace values MUST be valid item objects.
    add_op = create_model(
        "AddOp",
        op=(Literal["add"], "add"),
        path=(str, Field(..., description="JSON pointer path")),
        value=(item_model, Field(..., description="Item to add")),
    )
    replace_op = create_model(
        "ReplaceOp",
        op=(Literal["replace"], "replace"),
        path=(str, Field(..., description="JSON pointer path")),
        value=(item_model, Field(..., description="Full item replacement")),
    )
    remove_op = create_model(
        "RemoveOp",
        op=(Literal["remove"], "remove"),
        path=(str, Field(..., description="JSON pointer path")),
    )
    move_op = create_model(
        "MoveOp",
        op=(Literal["move"], "move"),
        from_path=(str, Field(..., description="Source path (use from_path)")),
        path=(str, Field(..., description="Target path")),
    )
    copy_op = create_model(
        "CopyOp",
        op=(Literal["copy"], "copy"),
        from_path=(str, Field(..., description="Source path (use from_path)")),
        path=(str, Field(..., description="Target path")),
    )
    patch_op = Annotated[
        Union[add_op, replace_op, remove_op, move_op, copy_op],
        Field(discriminator="op"),
    ]
    patch_plan = create_model(
        "PatchPlan",
        operations=(list[patch_op], Field(..., description="RFC6902 ops")),
    )
    return patch_plan


def make_request_model(patch_plan: Type[BaseModel]) -> Type[BaseModel]:
    class PatchRequest(BaseModel):
        mode: Literal["new", "patch_of_patch"]
        patch: patch_plan | JsonPatch

        @model_validator(mode="after")
        def _validate_mode(self):
            if self.mode == "new" and not isinstance(self.patch, patch_plan):
                raise ValueError("mode=new requires a full patch plan")
            if self.mode == "patch_of_patch" and not isinstance(self.patch, JsonPatch):
                raise ValueError("mode=patch_of_patch requires a JsonPatch")
            return self

    return PatchRequest


def _classify_error(exc: Exception, *, op: dict | None = None, op_index: int | None = None) -> ErrorInfo:
    message = str(exc)
    lower = message.lower()

    if "no previous patch" in lower:
        category = "patch_of_patch_missing"
    elif isinstance(exc, IndexError) or "out of range" in lower:
        category = "invalid_index"
    elif isinstance(exc, KeyError) or "invalid field" in lower or "cannot access nested field" in lower or "path" in lower:
        category = "invalid_path"
    elif "field required" in lower:
        category = "missing_field"
    elif "input should be" in lower or "literal" in lower or "enum" in lower:
        category = "invalid_enum"
    elif "patch resulted in invalid" in lower or "validation" in lower:
        category = "schema_validation"
    else:
        category = "unknown"

    return ErrorInfo(category=category, message=message, op=op, op_index=op_index)


class ObjectPatcher(Generic[T]):
    # General-purpose patcher for "model with one list field" using AutoGen tool calling.
    def __init__(
        self,
        model_cls: Type[T],
        *,
        model_client,
        initial_state: BaseModel | dict | None = None,
        list_field: str | None = None,
    ) -> None:
        self.model_cls = model_cls
        self.list_field = list_field or _infer_list_field(model_cls)
        self.item_model = _infer_item_model(model_cls, self.list_field)
        self.patch_plan = make_patch_schema(self.item_model)
        self.PatchRequest = make_request_model(self.patch_plan)
        self.patch_request_schema = self.PatchRequest.model_json_schema()
        self.ResultModel = PatchResult[model_cls]
        self.spec = ArtifactEditorToolSpec(model_cls)
        self._set_initial_state(initial_state)
        self._initial_snapshot = self._artifact()
        self._last_patch: BaseModel | None = None
        self._last_success: BaseModel | None = None

        self._agent = AssistantAgent(
            name="PatchAgent",
            model_client=model_client,
            tools=[self._build_tool()],
            max_tool_iterations=1,
            tool_call_summary_format="{result}",
            system_message=PATCH_SYSTEM_MESSAGE,
        )
        self._summarizer = AssistantAgent(
            name="PatchSummarizer",
            model_client=model_client,
            system_message=SUMMARY_SYSTEM_MESSAGE,
        )

    def _build_tool(self) -> FunctionTool:
        async def apply_patch(request: self.PatchRequest) -> PatchResult[T]:  # type: ignore[name-defined]
            return self._handle_request(request)

        return FunctionTool(
            apply_patch,
            description="Apply a patch request to the current artifact.",
            strict=True,
        )

    def _set_initial_state(self, initial_state: BaseModel | dict | None) -> None:
        if initial_state is None:
            data = {self.list_field: []}
        elif isinstance(initial_state, BaseModel):
            data = initial_state.model_dump()
        else:
            data = dict(initial_state)
        if self.list_field not in data:
            data[self.list_field] = []

        # Allow invalid inputs by constructing items without validation; patcher fixes them.
        if isinstance(self.item_model, type) and issubclass(self.item_model, BaseModel):
            items = []
            for item in data[self.list_field]:
                if isinstance(item, BaseModel):
                    items.append(item)
                else:
                    items.append(self.item_model.model_construct(**dict(item)))
            data[self.list_field] = items

        try:
            self.spec.current_artifact = self.model_cls.model_validate(data)
        except Exception:
            self.spec.current_artifact = self.model_cls.model_construct(**data)

    def _artifact(self) -> dict:
        return self.spec.get_current_artifact() or {self.list_field: []}

    def _handle_request(self, request: BaseModel) -> BaseModel:
        patch = request.patch
        if request.mode == "patch_of_patch":
            try:
                patch = self._apply_patch_to_patch(request.patch)
            except Exception as exc:
                error_info = _classify_error(exc)
                return self.ResultModel.model_validate(
                    {
                        "ok": False,
                        "result": self.model_cls.model_construct(**self._artifact()),
                        "current_artifact": self.model_cls.model_construct(**self._artifact()),
                        "applied": [],
                        "error": str(exc),
                        "summary": None,
                        "error_info": error_info,
                    }
                )
        self._last_patch = patch
        return self._apply_patch_sequential(patch)

    def _apply_patch_to_patch(self, patch_delta: JsonPatch) -> BaseModel:
        if self._last_patch is None:
            raise ValueError("No previous patch to repair.")
        editor = ArtifactEditorToolSpec(self.patch_plan)
        editor.current_artifact = self._last_patch
        updated = editor.apply_patch(patch_delta)
        return self.patch_plan.model_validate(updated)

    def _apply_patch_sequential(self, patch: BaseModel) -> PatchResult[T]:
        # Apply op-by-op so we can recover at the last successful patch.
        applied: list[dict] = []
        for op_index, op in enumerate(patch.operations):
            op_dict = op.model_dump(exclude_none=True)
            try:
                self.spec.apply_patch(JsonPatch.model_validate({"operations": [op_dict]}))
                applied.append(op_dict)
            except Exception as exc:
                error_info = _classify_error(exc, op=op_dict, op_index=op_index)
                return self.ResultModel.model_validate(
                    {
                        "ok": False,
                        "result": self.model_cls.model_construct(**self._artifact()),
                        "current_artifact": self.model_cls.model_construct(**self._artifact()),
                        "applied": applied,
                        "error": str(exc),
                        "summary": None,
                        "error_info": error_info,
                    }
                )
        result_model = self.model_cls.model_validate(self._artifact())
        self._last_success = result_model
        return self.ResultModel.model_validate(
            {
                "ok": True,
                "result": result_model,
                "current_artifact": result_model,
                "applied": applied,
                "error": None,
                "summary": None,
                "error_info": None,
            }
        )

    def _render_prompt(self, instruction: str, error: str | None, error_info: ErrorInfo | None) -> str:
        template = Template(PATCH_TEMPLATE)
        return template.render(
            list_field=self.list_field,
            artifact=json.dumps(self._artifact(), indent=2),
            instruction=instruction,
            item_schema=json.dumps(self.item_model.model_json_schema(), indent=2)
            if hasattr(self.item_model, "model_json_schema")
            else json.dumps({}, indent=2),
            patch_schema=json.dumps(self.patch_plan.model_json_schema(), indent=2),
            request_schema=json.dumps(self.patch_request_schema, indent=2),
            last_patch=json.dumps(self._last_patch.model_dump(), indent=2) if self._last_patch else "null",
            error_block=(f"Previous error: {error}" if error else ""),
            error_info=json.dumps(error_info.model_dump(), indent=2) if error_info else "null",
        )

    async def run(self, instruction: str, max_attempts: int = 3) -> PatchResult[T]:
        last_error: str | None = None
        last_error_info: ErrorInfo | None = None
        for _ in range(max_attempts):
            prompt = self._render_prompt(instruction, last_error, last_error_info)
            response = await self._agent.on_messages([TextMessage(content=prompt, source="user")], CancellationToken())
            try:
                outcome = self.ResultModel.model_validate_json(response.chat_message.content)
            except Exception as exc:
                last_error = f"Invalid tool result: {exc}"
                continue
            if outcome.ok:
                return outcome
            last_error = outcome.error
            last_error_info = outcome.error_info

        summary = await self._summarize_failure(last_error or "unknown", last_error_info)
        return self.ResultModel.model_validate(
            {
                "ok": False,
                "result": self.model_cls.model_construct(**self._artifact()),
                "current_artifact": self.model_cls.model_construct(**self._artifact()),
                "applied": [],
                "error": last_error,
                "summary": summary,
                "error_info": last_error_info,
            }
        )

    async def _summarize_failure(self, error: str, error_info: ErrorInfo | None) -> str:
        summary_prompt = Template(SUMMARY_TEMPLATE).render(
            error=error,
            error_info=json.dumps(error_info.model_dump(), indent=2) if error_info else "null",
            artifact=json.dumps(self._artifact(), indent=2),
            last_patch=json.dumps(self._last_patch.model_dump(), indent=2) if self._last_patch else "null",
        )
        response = await self._summarizer.on_messages([TextMessage(content=summary_prompt, source="user")], CancellationToken())
        return response.chat_message.content.strip()

    def restart_from(self, state: Literal["initial", "last_success"]) -> None:
        if state == "last_success" and self._last_success is not None:
            self._set_initial_state(self._last_success)
        else:
            self._set_initial_state(self._initial_snapshot)


In [14]:
model = globals().get("model") or "google/gemini-3-flash-preview"

PATCH_SYSTEM_MESSAGE = """
You are a patching agent. Your job is to call the apply_patch tool with a PatchRequest.
Do not answer with prose. Always call the tool.
""".strip()

PATCH_TEMPLATE = """
You are editing a list in a JSON artifact.
Goal: make the list match the user's instruction.

Rules:
- Use ONLY the apply_patch tool.
- Choose mode = "new" for a fresh patch plan.
- Choose mode = "patch_of_patch" to repair the last patch plan.
- Operations are applied sequentially; indices refer to the current state.
- For add/replace, value MUST be a full item matching the item schema.
- Allowed ops: add, replace, remove, move, copy.
- Use from_path (not from) for move/copy.
- Prefer small, minimal patches.
- Replace whole list items (use path like /{{list_field}}/0), not subfields.

List field: /{{list_field}}

Current artifact:
{{artifact}}

User instruction:
{{instruction}}

Item schema:
{{item_schema}}

Patch schema:
{{patch_schema}}

PatchRequest schema:
{{request_schema}}

Last patch (for patch_of_patch mode):
{{last_patch}}

Error info (structured):
{{error_info}}

{{error_block}}
""".strip()

SUMMARY_SYSTEM_MESSAGE = """
You are a concise failure summarizer. Respond with 2-3 bullets.
""".strip()

SUMMARY_TEMPLATE = """
Summarize the patch failure in 2-3 bullets and suggest next steps.

Error: {{error}}
Error info: {{error_info}}
Current artifact: {{artifact}}
Last patch: {{last_patch}}
""".strip()


# Patch walkthrough (AutoGen + ArtifactEditor)

**Purpose:** Patch an *incorrect* list-based object into the desired state using **AutoGen tool calling** + **typed JSON Patch**.

## Steps (mapped to code)
1. **Infer list + item types**: `_infer_list_field` + `_infer_item_model` find the list field and its item type.  
2. **Typed patch schema**: `make_patch_schema(item_model)` forces `add/replace.value` to be valid items.  
3. **PatchRequest schema**: `make_request_schema` lets the agent choose **new** patch or **patch_of_patch**.  
4. **Tool call**: `ObjectPatcher._build_tool` exposes `apply_patch(request)` as an AutoGen tool (strict schema).  
5. **Sequential apply**: `_apply_patch_sequential` applies op-by-op for recoverability.  
6. **Repair loop**: `run` retries; on failure, summarize and allow restart options.

## Trade-offs
- **Pros:** schema enforcement *before* patching, AutoGen tool loop, recoverable failures.
- **Cons:** assumes one list field per model; sequential patching is slower but safer for index shifts.


# Mermaid diagrams

```mermaid
flowchart TD
  A[User instruction] --> B[ObjectPatcher.run]
  B --> C[_render_prompt]
  C --> D[AutoGen AssistantAgent]
  D --> E[apply_patch tool]
  E --> F[ArtifactEditorToolSpec.apply_patch]
  F --> G[Updated artifact]
```

```mermaid
sequenceDiagram
  participant U as User
  participant P as ObjectPatcher
  participant A as AutoGen Agent
  participant T as apply_patch tool
  participant E as ArtifactEditor

  U->>P: instruction + initial_state
  P->>A: prompt (artifact + schemas)
  A->>T: PatchRequest(mode=new|patch_of_patch)
  T->>E: apply patch ops sequentially
  E-->>T: updated artifact or error
  T-->>A: PatchOutcome (JSON)
  A-->>P: tool summary (JSON)
```


# TODOs
- [ ] Allow models with multiple list fields (explicit list_field required).
- [ ] Add a small "patch preview" mode (no apply, just diff).
- [ ] Expand repair prompts with structured error categories.


In [17]:
wrong_pizza_order = {
    "pizzas": [
        {"style": "napolitaon", "toppings": "ananas", "size": "m", "quantity": 1}
    ]
}

model_client = build_autogen_chat_client("patch_agent", model=model)
patcher = ObjectPatcher(PizzaOrder, model_client=model_client, initial_state=wrong_pizza_order)

instruction = (
    "Make it a napoletana, size l, toppings pepperoni and mushrooms, quantity 2. "
    "Add a second pizza: sicilian, size s, toppings olives, quantity 1."
)

result = await patcher.run(instruction, max_attempts=3)
result


PatchResult[PizzaOrder](ok=True, result=PizzaOrder(pizzas=[Pizza(style='napoletana', toppings='pepperoni and mushrooms', size='l', quantity=2), Pizza(style='sicilian', toppings='olives', size='s', quantity=1)]), current_artifact=PizzaOrder(pizzas=[Pizza(style='napoletana', toppings='pepperoni and mushrooms', size='l', quantity=2), Pizza(style='sicilian', toppings='olives', size='s', quantity=1)]), applied=[{'op': 'replace', 'path': '/pizzas/0', 'value': {'style': 'napoletana', 'toppings': 'pepperoni and mushrooms', 'size': 'l', 'quantity': 2}}, {'op': 'add', 'path': '/pizzas/1', 'value': {'style': 'sicilian', 'toppings': 'olives', 'size': 's', 'quantity': 1}}], error=None, summary=None, error_info=None)

In [16]:
result

PatchResult[PizzaOrder](ok=True, result=PizzaOrder(pizzas=[Pizza(style='napoletana', toppings='pepperoni and mushrooms', size='l', quantity=2), Pizza(style='sicilian', toppings='olives', size='s', quantity=1)]), current_artifact=PizzaOrder(pizzas=[Pizza(style='napoletana', toppings='pepperoni and mushrooms', size='l', quantity=2), Pizza(style='sicilian', toppings='olives', size='s', quantity=1)]), applied=[{'op': 'replace', 'path': '/pizzas/0', 'value': {'style': 'napoletana', 'toppings': 'pepperoni and mushrooms', 'size': 'l', 'quantity': 2}}, {'op': 'add', 'path': '/pizzas/1', 'value': {'style': 'sicilian', 'toppings': 'olives', 'size': 's', 'quantity': 1}}], error=None, summary=None, error_info=None)